![min](http://www.muslimheritage.com/sites/default/files/east_meets_west_venice_02_1.jpg?slideshow=true&slideshowAuto=false&slideshowSpeed=4000&speed=350&transition=elastic)

Object detection is very popular field of interest and new and new algorithms for improving it are realised by the day. In Airbus Ship Detection Challege the accent is on speed (of course not neglecting the accuracy too) which makes problem even more complicated. When working on such kind of models people often need to do some Image Processing first.
### The idea of this Kernel is to do some feature engineering on images before feeding it to machine learning algorithms.  
With this in mind I performed several image processing techniques and went lightly through them, showing the joined results in the end. This could be good start before building predicting algorithm.

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from skimage import io
from skimage import measure 
from skimage import feature
from skimage.morphology import disk
from skimage.filters import rank
from skimage import exposure
import warnings
warnings.filterwarnings("ignore")

In [ ]:
marks = pd.read_csv('../input/airbus-ship-detection/train_ship_segmentations.csv') # Markers for ships
files = os.listdir('../input/airbus-ship-detection/train') # Images for training
ship = io.imread('../input/picture/picture.png') 
os.chdir("../input/airbus-ship-detection/train")

In [ ]:
Img_Height = 768
Img_Width = 768

In [ ]:
def mask_part(pic):
    '''
    Function that converts single marker from 'marks' into the image
    '''
    back = np.zeros(Img_Height**2)
    starts = pic.split()[0::2]
    lens = pic.split()[1::2]
    for i in range(len(lens)):
        back[(int(starts[i])-1):(int(starts[i])-1+int(lens[i]))] = 1
    return np.reshape(back, (Img_Height, Img_Width))

In [ ]:
def is_empty(key):
    '''
    Checks if there is a marker for specific image
    i.e. if there is a ship on image
    '''
    df = marks[marks['ImageId'] == key].iloc[:,1]
    if len(df) == 1 and type(df.iloc[0]) != str and np.isnan(df.iloc[0]):
        return True
    else:
        return False

In [ ]:
def masks_all(key):
    '''
    Collects together all single markers belonging to the same image
    '''
    df = marks[marks['ImageId'] == key].iloc[:,1]
    masks= np.zeros((Img_Height, Img_Width))
    if is_empty(key):
        return masks
    else:
        for i in range(len(df)):
            masks += mask_part(df.iloc[i])
        return masks.T

In [ ]:
def draw(file, rgb=0):
    '''
    Draws original image, correspondin mask and sum of two'''
    plt.figure(figsize = (15,10))
    plt.subplot(131, title ='Original Image')
    plt.imshow(plt.imread(file))
    plt.axis('off')
    plt.subplot(132, title ='Mask of an Image')
    plt.imshow(masks_all(file))
    plt.axis('off')
    plt.subplot(133, title ='Combined') 
    plt.imshow(plt.imread(file)[:,:,rgb]+masks_all(file)*200)
    plt.axis('off')
    plt.suptitle(file, y=0.77, verticalalignment ='top', fontsize = 22)
    plt.show()
    print('')

In [ ]:
ships = [] # Here I restrict images only on those containing ship, for the sake of visualization
for file in files:
    if not is_empty(file):
        ships.append(file)

### Below are 20 randomly chosen images with corresponding masks
As you can see from these plots, it is often very hard task even for you to detect a ship (or all the ships). So, you'll need a hell of a good model to achieve good results. 

In [ ]:
for file in np.random.choice(ships, 20):
    draw(file, rgb=1)

## Finding Contours
It maght be interesting to look for the contours on image, becouse they could be around the ship and make problem a little easier.
This is not so trivial, becouse parameter must be choosen differently for different images so you need functions that will pick it wisely. The 'Parameter' function below tries to deal with this problem in a somewhat naive way, looking only if parameter is feasible, but not iff it gives good result.

In [ ]:
def Parameter(file):
    p = np.arange(10,250,20)
    for i in p:
        if Param(file, i):
            break
    return i

In [ ]:
def Param(file, param):
    '''
    Chooses feasible parameter for the function Contour (below)'''
    img = io.imread(file)
    img0 = img[:,:,0]
    contours0 = measure.find_contours(img0, param)
    img1 = img[:,:,1]
    contours1 = measure.find_contours(img1, param)
    img2 = img[:,:,2]
    contours2 = measure.find_contours(img2, param)
    if len(contours0) == 0 or len(contours1)==0 or len(contours2)==0:
        return False
    else:
        return True

In [ ]:
def Contour(file, param=150):
    '''
    Looks for contours on images across all three channels
    '''
    img = io.imread(file)
    img0 = img[:,:,0]
    contours0 = measure.find_contours(img0, param)
    contour0 = max(contours0, key=len)
    img1 = img[:,:,1]
    contours1 = measure.find_contours(img1, param)
    contour1 = max(contours1, key=len)
    img2 = img[:,:,2]
    contours2 = measure.find_contours(img2, param)
    contour2 = max(contours2, key=len)
    return contour0, contour1, contour2

In [ ]:
def Draw_contour(file, param=150):
    '''
    Plots together contour on an original image,
    zoomed part of original image containing contoure and contoure alone
    in each of three color channels
    '''
    img = io.imread(file)
    img0, img1, img2 = img[:,:,0],img[:,:,1],img[:,:,2]
    contour0, contour1, contour2 = Contour(file, param)
    fig, (ax, ax3, ax2) = plt.subplots(ncols=3, figsize=(20, 10))
    fig.figsize = (20,20)
    ax.plot(contour0[::,1], contour0[::,0], color = 'g', linewidth = 0.2)
    ax.imshow(img0, origin='lower', cmap='Reds')
    ax.axis('off')
    ax.set(title = 'Red Spectrum')
    ax2.plot(contour0[::,1], contour0[::,0], color = 'tan')
    asp = np.diff(ax2.get_xlim())[0] / np.diff(ax2.get_ylim())[0]
    ax2.set_aspect(asp)
    ax2.axis('off')
    ax2.set(title = 'Only Contour')
    ax3.set(xlim=(min(contour0[::,1])-50,max(contour0[::,1])+50), ylim=(min(contour0[::,0])-50,max(contour0[::,0])+50), autoscale_on=False,
               title='Zoom Section with Contour')
    ax3.plot(contour0[::,1], contour0[::,0], color = 'g', linewidth = 0.9)
    ax3.imshow(img0, cmap='Reds')
    ax3.axis('off')
    plt.suptitle(file, y=0.85, fontsize = 22, x=0.25)
    plt.show()

    fig, (ax, ax3, ax2) = plt.subplots(ncols=3, figsize=(20, 10))
    fig.figsize = (20,20)
    ax.plot(contour1[::,1], contour1[::,0], color = 'r', linewidth = 0.2)
    ax.imshow(img1, origin='lower', cmap='Greens')
    ax.axis('off')
    ax.set(title = 'Green Spectrum')
    ax2.plot(contour1[::,1], contour1[::,0], color = 'tan')
    asp = np.diff(ax2.get_xlim())[0] / np.diff(ax2.get_ylim())[0]
    ax2.set_aspect(asp)
    ax2.axis('off')
    ax2.set(title = 'Only Contour')
    ax3.set(xlim=(min(contour1[::,1])-50,max(contour1[::,1])+50), ylim=(min(contour1[::,0])-50,max(contour1[::,0])+50), autoscale_on=False,
               title='Zoom Section with Contour')
    ax3.plot(contour1[::,1], contour1[::,0], color = 'r', linewidth = 0.9)
    ax3.imshow(img1, cmap='Greens')
    ax3.axis('off')
    plt.suptitle(file, y=0.85, fontsize = 22, x=0.25)
    plt.show()

    fig, (ax, ax3, ax2) = plt.subplots(ncols=3, figsize=(20, 10))
    fig.figsize = (20,20)
    ax.plot(contour2[::,1], contour2[::,0], color = 'r', linewidth = 0.2)
    ax.imshow(img2, origin='lower', cmap='Blues')
    ax.axis('off')
    ax.set(title = 'Blue Spectrum')
    ax2.plot(contour2[::,1], contour2[::,0], color = 'tan')
    asp = np.diff(ax2.get_xlim())[0] / np.diff(ax2.get_ylim())[0]
    ax2.set_aspect(asp)
    ax2.axis('off')
    ax2.set(title = 'Only Contour')
    ax3.set(xlim=(min(contour2[::,1])-20,max(contour2[::,1])+20), ylim=(min(contour2[::,0])-20,max(contour2[::,0])+20), autoscale_on=False,
               title='Zoom Section with Contour')
    ax3.plot(contour2[::,1], contour2[::,0], color = 'r', linewidth = 0.9)
    ax3.imshow(img2, cmap='Blues')
    ax3.axis('off')
    plt.suptitle(file, y=0.85, fontsize = 22, x=0.25)
    plt.show()
    print('')

### Plotting contours through color channels
From the plots below we can see how contour on the same image can differ widely depending on spectrum. Sometimes this feature seems to be really helpful pointing to the whole ship or parts of it, but unfortunately, more offten it corresponds to the coast line or the border of a cloud. Even so, it might be of use to detect the cosat line when looking for ship. Anyway, with some better parameter tuning maybe results would be better too.

In [ ]:
for file in np.random.choice(ships, 20):
    Draw_contour(file, Parameter(file))

## Equalizing and Stretching Image Contrast
Lowering contrast is common in image processing, but also often used in order to 'nicer' looking picture, 
presenting in a way we consider looking good. On the other side, contrast stretching creates pictures that
aren't so 'nice' but are probably better suited for problems like this where you want to detect object.
Both these techniques are shown below.

In [ ]:
def Equalizer(file):
    '''
    Equalizes image spectrum (in gray scale)
    '''
    img = io.imread(file, as_grey=True)
    selem = disk(100)
    img_eq = rank.equalize(img, selem=selem)
    return img_eq

In [ ]:
for file in np.random.choice(ships, 10):
    plt.imshow(Equalizer(file), cmap='inferno')
    plt.axis('off')
    plt.show()

In [ ]:
def Contrast(file, channel = None):
    '''
    Creates high contrast image (in specified channel)
    '''
    img = io.imread(file, as_grey= False)
    left, right = np.percentile(img, (30, 70))
    if channel ==None:
        img_rescale = exposure.rescale_intensity(img, in_range=(left, right))
    else:
        img_rescale = exposure.rescale_intensity(img[:,:,channel], in_range=(left, right))
    return img_rescale

In [ ]:
def draw_contrast(file):
    '''
    Plots together original image, 
    high contrast image (in rgb and in each of color channels)
    and lower contrast image
    '''
    img = io.imread(file) 
    print('')
    plt.figure(figsize = (15,15))
    plt.subplot(161, title ='Original')
    plt.imshow(img)
    plt.axis('off')
    plt.subplot(162, title ='High Contrast (HC)')
    plt.imshow(Contrast(file))
    plt.axis('off')
    plt.subplot(163, title ='HC Red Spectrum') 
    plt.imshow(Contrast(file, 0), cmap='inferno')
    plt.axis('off')
    plt.subplot(164, title ='HC Green Spectrum') 
    plt.imshow(Contrast(file, 1), cmap='inferno')
    plt.axis('off')
    plt.subplot(165, title ='HC Blue Spectrum') 
    plt.imshow(Contrast(file, 2), cmap='inferno')
    plt.axis('off')
    plt.subplot(166, title ='Equalized Image') 
    plt.imshow(Equalizer(file), cmap='inferno')
    plt.axis('off')
    plt.suptitle(file, y=0.63, fontsize = 22)
    plt.show()      

In [ ]:
for file in np.random.choice(ships, 20):
    draw_contrast(file)

In [ ]:
def plot_all(file):
    '''
    Plots all extracted features together
    '''
    img = io.imread(file)
    plt.figure(figsize = (12,15))
    plt.subplot(431)
    plt.xlim(4000,0)
    plt.imshow(ship)
    plt.axis('off')
    plt.subplot(432, title ='Original')
    plt.imshow(img)
    plt.axis('off')
    plt.subplot(433)
    plt.imshow(ship)
    plt.axis('off')
    plt.subplot(434, title ='Mask')
    plt.imshow(masks_all(file))
    plt.axis('off')
    plt.subplot(435, title ='High Contrast (HC)')
    plt.imshow(Contrast(file))
    plt.axis('off')
    plt.subplot(436, title ='Equalized Image') 
    plt.imshow(Equalizer(file), cmap='inferno')
    plt.axis('off')
    plt.subplot(437, title ='HC Red Spectrum') 
    plt.imshow(Contrast(file, 0), cmap='inferno')
    plt.axis('off')
    plt.subplot(438, title ='HC Green Spectrum') 
    plt.imshow(Contrast(file, 1), cmap='inferno')
    plt.axis('off')
    plt.subplot(439, title ='HC Blue Spectrum') 
    plt.imshow(Contrast(file, 2), cmap='inferno')
    plt.axis('off')
    c0, c1, c2 = Contour(file, param=Parameter(file))
    plt.subplot(4,3,10, title ='Red Spectrum Contours') 
    plt.plot(c0[::,1], c0[::,0], color = 'tan')
    plt.axis('off')
    plt.imshow(np.zeros((768,768)), cmap = 'summer')
    plt.subplot(4,3,11, title ='Green Spectrum Contours') 
    plt.plot(c0[::,1], c0[::,0], color = 'tan')
    plt.axis('off')
    plt.imshow(np.zeros((768,768)), cmap = 'summer')
    plt.subplot(4,3,12, title ='Blue Spectrum Contours') 
    plt.plot(c0[::,1], c0[::,0], color = 'tan')
    plt.axis('off')
    plt.imshow(np.zeros((768,768)), cmap ='summer')
    plt.suptitle(file, y=0.92, fontsize = 22)
    plt.show()
    print('')

## All Together
We have come to the end, and below are presented together all features extracted. Hope this was usefull

In [ ]:
for file in np.random.choice(ships, 20):
    plot_all(file)

##### That's all folks